## LIGN 167: Problem Set 4
### Name: Chih-Hsuan Kao
### Date: December 3, 2018

An implementation of a simple RNN (an Elman network) in PyTorch.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import spacy


###### DATA PROCESSING STARTS #########


def load_corpus():
    #this loads the data from sample_corpus.txt
    with open('/Users/carolynkao/Desktop/LIGN167/psa5/sample_corpus.txt','r') as f:
        corpus = f.read().replace('\n','')
    return corpus
corpus = load_corpus()
corpus

In [49]:
def segment_and_tokenize(corpus):
    #make sure to run: python -m spacy download en 
    #in the command line before using this!

    #corpus is assumed to be a string, containing the entire corpus
    nlp = spacy.load('en')
    tokens = nlp(corpus)
    sents = [[t.text for t in s] for s in tokens.sents if len([t.text for t in s])>1]
    sents = remove_infrequent_words(sents)
    sents = [['<START>']+s+['<END>'] for s in sents]
    return sents

sents2 = segment_and_tokenize(load_corpus())
sents2

[['<START>',
  '<UNKOWN>',
  ',',
  'baby',
  'animals',
  'who',
  'are',
  '<UNKOWN>',
  'by',
  'their',
  'parents',
  'are',
  'no',
  'longer',
  'kept',
  'in',
  'the',
  '<UNKOWN>',
  "'s",
  '<UNKOWN>',
  'because',
  'the',
  '<UNKOWN>',
  'has',
  'found',
  'that',
  'it',
  'is',
  '<UNKOWN>',
  'to',
  '<UNKOWN>',
  'them',
  'into',
  'their',
  '<UNKOWN>',
  'once',
  'they',
  '<UNKOWN>',
  'up',
  'if',
  'they',
  'are',
  'kept',
  'away',
  'from',
  'their',
  'own',
  'species',
  '.Some',
  '<END>'],
 ['<START>',
  'people',
  'think',
  'that',
  'before',
  'becoming',
  'a',
  'god',
  ',',
  'he',
  'was',
  'a',
  'Chinese',
  '<UNKOWN>',
  'and',
  'a',
  '<UNKOWN>',
  'of',
  'a',
  '<UNKOWN>',
  'god',
  '.The',
  '<END>'],
 ['<START>',
  'Grand',
  'Canyon',
  'is',
  'a',
  'very',
  'large',
  '<UNKOWN>',
  'in',
  'Arizona',
  ',',
  'United',
  'States',
  '.',
  '<END>'],
 ['<START>',
  'It',
  'is',
  'a',
  'national',
  'park',
  'of',
  'the',

In [50]:
def remove_infrequent_words(sents):
    word_counts = {}
    for s in sents:
        for w in s:
            if w in word_counts:
                word_counts[w] += 1
            else:
                word_counts[w] = 1

    threshold = 2
    filtered_sents = []
    for s in sents:
        new_s = []
        for w in s:
            if word_counts[w] < threshold:
                new_s.append('<UNKOWN>')
            else:
                new_s.append(w)
        filtered_sents.append(new_s)
    return filtered_sents



def make_word_to_ix(sents):
    word_to_ix = {}
    num_unique_words = 0
    for sent in sents:
        for word in sent:
            if word not in word_to_ix:
                word_to_ix[word] = num_unique_words
                num_unique_words += 1


    return word_to_ix

def sent_to_onehot_vecs(sent,word_to_ix):
    #note: this is not how you would do this in practice! 

    vecs = []
    for i in range(len(sent)):
        word = sent[i]
        word_index = word_to_ix[word]

        vec = torch.zeros(len(word_to_ix), dtype=torch.float32,requires_grad=False)
        vec[word_index] = 1
        vecs.append(vec)

    return vecs

def vectorize_sents(sents,word_to_ix):
    one_hot_vecs = []
    for s in sents:
        one_hot_vecs.append(sent_to_onehot_vecs(s,word_to_ix))
    return one_hot_vecs

def get_data():
    corpus = load_corpus()
    sents = segment_and_tokenize(corpus)
    word_to_ix = make_word_to_ix(sents)

    vectorized_sents = vectorize_sents(sents,word_to_ix)
    
    vocab_size = len(word_to_ix)

    return vectorized_sents, vocab_size

sents=[['The','dog','barked'],['The','cat','barked']]
word_to_ix = make_word_to_ix(sents2)
vectorized_sents = vectorize_sents(sents2,word_to_ix)
print(len(sents2))
print(len(word_to_ix))
print(len(vectorized_sents))
###### DATA PROCESSING ENDS #########

678
1682
678


In [51]:
###### RNN DEFINITION STARTS #########

def elman_unit(word_embedding,h_previous,W_x,W_h,b):
    return torch.sigmoid(torch.matmul(W_x,word_embedding)+torch.matmul(W_h,h_previous)+b)

def embed_word(word,W_e):
    #word is a one-hot vector
    return torch.matmul(W_e,word)



def single_layer_perceptron(h,W_p):
    s = torch.matmul(W_p,h)
    softmax = nn.Softmax(dim=0)
    return softmax(s)


def network_forward(sent,param_dict):
    W_e = param_dict['W_e']
    W_x = param_dict['W_x'] 
    W_h = param_dict['W_h']
    W_p = param_dict['W_p']
    b = param_dict['b']


    h_previous = initialize_hidden_state(W_h.size()[1])
    
    predictions = []
    for i in range(len(sent)-1):
        current_word = sent[i]

        current_word_embedding = embed_word(current_word,W_e)

        h_current = elman_unit(current_word_embedding,h_previous, W_x,W_h,b)

        prediction = single_layer_perceptron(h_current,W_p)
        predictions.append(prediction)

        h_previous = h_current

    return predictions



###### RNN DEFINITION ENDS #########


In [52]:
#### LOSS FUNCTION BEGINS #######

def word_loss(word_probs, word):
    #outcome is a one-hot vector
    prob_of_word = torch.dot(word_probs,word)
    return -1*torch.log(prob_of_word)

def sent_loss(predictions, sent):
    L = torch.tensor(0,dtype=torch.float32)

    for i in range(len(predictions)):
        word_probs = predictions[i]
        observed_word = sent[i+1]
        L+=word_loss(word_probs,observed_word)

    return L


##### LOSS FUNCTION ENDS #######



In [53]:
#####WEIGHT INITIALIZATION STARTS #######

def initialize_weight_matrix(shape):
    return torch.rand(shape,dtype=torch.float32,requires_grad=True)

def initialize_hidden_state(shape):
    return torch.zeros(shape,dtype=torch.float32,requires_grad=False)


def initialize_parameters(embedding_dim,vocab_size,hidden_state_dim):
    W_e, W_x, W_h,W_p, b = (initialize_weight_matrix([embedding_dim,vocab_size]), 
        initialize_weight_matrix([hidden_state_dim,embedding_dim]), 
        initialize_weight_matrix([hidden_state_dim,hidden_state_dim]),
        initialize_weight_matrix([vocab_size,hidden_state_dim]),
        initialize_weight_matrix(hidden_state_dim))


    param_dict = {}
    param_dict['W_e'] = W_e
    param_dict['W_x'] = W_x
    param_dict['W_h'] = W_h
    param_dict['W_p'] = W_p
    param_dict['b'] = b

    return param_dict


#####WEIGHT INITIALIZATION ENDS #######




In [54]:
def train():
        
    vectorized_sents, vocab_size = get_data()
    

    num_epochs = 100

    hidden_state_dim = 10
    embedding_dim = 10
    learning_rate = 0.00001

    


    param_dict = initialize_parameters(embedding_dim,vocab_size,hidden_state_dim)

    optimizer = optim.SGD(list(param_dict.values()), lr=learning_rate)

    for i in range(num_epochs):

        total_loss = 0
        for s in vectorized_sents:
            optimizer.zero_grad()
            predictions = network_forward(s,param_dict)
            loss = sent_loss(predictions,s)
            total_loss += loss

            loss.backward() #this is the crucial step! this is where PyTorch automatically calculates gradients for all of the model parameters
            optimizer.step() #this is where gradient descent occurs: for each weight vector w, w = w-lr*w.grad
        print(total_loss)


if __name__=='__main__':
    train()



tensor(142693.0938, grad_fn=<ThAddBackward>)
tensor(140887.8125, grad_fn=<ThAddBackward>)
tensor(139088.6719, grad_fn=<ThAddBackward>)
tensor(137297.3125, grad_fn=<ThAddBackward>)
tensor(135517.2031, grad_fn=<ThAddBackward>)
tensor(133752.4062, grad_fn=<ThAddBackward>)
tensor(132009.4531, grad_fn=<ThAddBackward>)
tensor(130296.3984, grad_fn=<ThAddBackward>)
tensor(128624.8672, grad_fn=<ThAddBackward>)
tensor(127009.3984, grad_fn=<ThAddBackward>)
tensor(125468.3828, grad_fn=<ThAddBackward>)
tensor(124022.6875, grad_fn=<ThAddBackward>)
tensor(122693.7031, grad_fn=<ThAddBackward>)
tensor(121499.9297, grad_fn=<ThAddBackward>)
tensor(120452.6016, grad_fn=<ThAddBackward>)
tensor(119552.5000, grad_fn=<ThAddBackward>)
tensor(118788.4766, grad_fn=<ThAddBackward>)
tensor(118140.7109, grad_fn=<ThAddBackward>)
tensor(117584.7344, grad_fn=<ThAddBackward>)
tensor(117096.5078, grad_fn=<ThAddBackward>)
tensor(116655.4219, grad_fn=<ThAddBackward>)
tensor(116246.2188, grad_fn=<ThAddBackward>)
tensor(115

# Written Section

## Problem 1. 
The function load_corpus loads the text from sample_corpus.txt, and returns it as a string.
The function segment_and_tokenize takes a single argument, corpus, which is assumed to be a string containing the entire corpus. Test this code on some simple examples, or on the corpus that is loaded by load_corpus. What does the function do to the string that it receives?

<font color='brown'>According to the corpus that it receives, the function segment_and_tokenize basically divides the passage into single tokens by space. Firstly, it loads the corpus and creates tokens. Notably, it removes infrequent words and returns $<UNKNOWN>$, then attaches $<START>$ in the beginning and $<END>$ in the end of each sentence. Generally, each sentence is divided based on the occurrence of period. The variable ‘sents’ is a list of sentences, where each sentence is itself a list of words. Finally, a list of frequently used words are returned.</font>


## Problem 2
The function get_data first loads the corpus, then creates a variable sents by calling segment_and_tokenize. It then passes sents to a function make_word_to_ix. The variable sents is assumed to be a list of sentences, where each sentence is itself a list of words. Explain what make_word_to_ix is doing.
Hint: Try this on some simple examples. For example, set sents equal to: [[’The’,’dog’,’barked’],[’The’,’cat’,’barked’]].


<font color='brown'>This function takes in a list of sentences, and reads in all words in the sentences. It then builds a dictionary of individual words corresponding to unique indexes. This dictionary does not allow duplicated key, including $<UNKNOWN>$, $<START>$ and $<END>$. </font>

## Problem 3. 
After the function get_data calls make_word_to_ix, it then calls a function vectorize_sents with arguments sents and word_to_ix.
vectorize_sents turns the sentences into one-hot vectors. Explain how it does this.

<font color='brown'> The function called vectorize_sents turns the sentences into one-hot vectors. The thing returned is a list, in which lists of sentences are contained. The list of sentences include lists of one-hot vector words. Firstly, the function takes in sents and word_to_ix. Then, it calls function sent_to_onehot_vecs for each sentence in sents. Then, sent_to_onehot_vecs would convert each word in that sentence into one hot vector by using word_to_ix. This is how one hot vectors are created.</font>

## Problem 4. 
The RNN defined in this code is being used for language modeling. As discussed in class, a language model is a probability distribution over sentences. If a sentence s consists of a sequence of words $w_0,...,w_{n−1}$, then the probability of the sentence is defined by:

$ P(s) = P(< start >,w_0,...,w_{n−1},< end >)
= P (w_0| < start >) · P (w_1| < start >, w_0) · ... · P (< end > | < start >, w_0, ..., w_{n−1}) $

We will walk through the steps that the RNN is using to define a language model. The main function in this part of the code is network_forward. This function takes two arguments: sent and param_dict. The argument sent is a list of one-hot vectors, each representing a single word in the sentence. The argument param_dict is a dictionary containing all of the parameters needed to define the RNN. These are the parameters that will be learned later on.
One of the first things that this function does is call embed_word, given a word (current_word) from the sentence and the weight matrix We. The function embed_word returns a word em- bedding for the word. How does embed_word generate a word embedding for current_word?

<font color='brown'>The function embed_word takes in two parameters: current_word, which is a word and the weight matrix w_e. This function performs matrix multiplication. The result gives out the word embedding for the passed-in current_word.</font>

## Problem 5. 
After generating the wordembedding forcurrent_word,the functionnetwork_forward then calls the function elman_unit. This function will apply an Elman unit (as discussed in class) to two inputs: current_word_embedding and h_previous (the hidden state at the previous time point). The function elman_unit takes five arguments in total: current_word_embedding, h_previous, and three weight matrices.

Describe mathematically what function elman_unit computes on its inputs.

Hint: torch.matmul performs matrix multiplication: it multiplies a matrix by a vector. There is very good documentation for torch.matmul, and all other PyTorch functions, that can be found through Google.

<font color='brown'>
$ v = w_x * word\_embedding + w_h * h\_previous + b $ <br>

$ v = [v_1, v_2, ..., v_m] $<br>

$ \sigma(v) = [\sigma(v_1), \sigma(v_2), ..., \sigma(v_m)] $ where $ \sigma $ is the sigmoid function, defined as $ \sigma(x)$ = $\frac{1}{1+e^{-x}} $

</font>

## Problem 6.
After calling elman_unit and generating the value h_current, the function network_forward then calls the function single_layer_perceptron. The function single_layer_perceptron takes two arguments: h_current and the weight matrix Wp. Describe mathematically what the function single_layer_perceptron computes on its inputs.

<font color='brown'> Let $ S = w_p * h = [S_1, S_2, ..., S_k] $ where $ W_p $ is the weight matrix <br>
Define softmax as $a(x)$, then $a(x)_j = \frac{e^{x_j}}{\sum_{i=1}^{k} e^{x_i}}$
Then, the single_layer_perceptron returns <br>
$a(S) = [a(s)_1, a(s)_2, ..., a(s)_k] $ <br>
$ = [\frac{e^{s_1}}{\sum_{i=1}^{k} e^{S_k}}, \frac{e^{s_2}}{\sum_{i=1}^{k} e^{S_k}}, ..., \frac{e^{s_k}}{\sum_{i=1}^{k} e^{S_k}} ] $ </font>

## Problem 7. 
The previous three problems have gone through the code describing the RNN. The function train is the code that is used for training the RNN. It first defines the dimen- sions for the different weight vectors, and then initializes the parameters. It uses PyTorch code for computing gradients automatically and optimizing the model weights.
When you run the code by calling train, what happens to the loss function over time? Name one problem that the current implementation of the RNN is likely to have. We discussed several in class.
Hint: What can go wrong with Elman networks?

<font color='brown'>The Loss Function decreases over time.

The RNN function would possibly return a tiny gradient which would cause the RNN to forget the start of the sentence.

Mathematically speaking, according to the vanishing gradient problem in class,</font>

<font color='brown'>$ \frac{\partial L}{\partial h_1} = \frac{\partial L}{\partial h_n} * (w_h)^{n-1} $ </font>

<font color='brown'>When $w_h$ is small, the latter term would be small and vice versa. <br>
Large gradient, on the other hand, would lead to the missing of global minimum.</font>